In [1]:
!pip install -r /content/requirements.txt

  Cloning https://github.com/Joeclinton1/google-images-download.git to /tmp/pip-req-build-wrrv05lo
  Running command git clone -q https://github.com/Joeclinton1/google-images-download.git /tmp/pip-req-build-wrrv05lo
     |████████████████████████████████| 4.2 MB 8.9 MB/s 
     |████████████████████████████████| 2.3 MB 43.9 MB/s 
     |████████████████████████████████| 979 kB 40.7 MB/s 
     |████████████████████████████████| 596 kB 48.8 MB/s 
     |████████████████████████████████| 6.6 MB 55.2 MB/s 
     |████████████████████████████████| 84 kB 4.0 MB/s 
     |████████████████████████████████| 79 kB 10.0 MB/s 
     |████████████████████████████████| 231 kB 49.6 MB/s 
     |████████████████████████████████| 51 kB 7.2 MB/s 
     |████████████████████████████████| 1.2 MB 44.2 MB/s 
     |████████████████████████████████| 359 kB 61.9 MB/s 
     |████████████████████████████████| 138 kB 61.6 MB/s 
     |████████████████████████████████| 138 kB 57.7 MB/s 
     |██████████████████████████████

In [5]:
article = """ Anthony Albanese is set to become Australia's next prime minister after leading the Labor Party to its first election victory in more than a decade.
One of the country's longest-serving politicians, he promised voters "safe change" as he worked to kick out the conservative Liberal-National coalition which has been in power since 2013.
Where ousted Prime Minister Scott Morrison is a self-described "bulldozer", Mr Albanese vows to be a "builder".
After the pandemic, which saw Australian states cut off from each other and cities splintered by strict lockdowns, fostering unity is a top priority for the new leader.
"I want to bring Australians together. I want to seek our common purpose and promote unity and optimism, not fear and division," he said in a victory speech on Saturday night.
So who is Albo?
Mr Albanese has earned a reputation as a defender of Australia's free healthcare system, an advocate for the LGBT community, a republican, and a passionate rugby league fan.The 59-year-old - nicknamed Albo - was raised in social housing by a single mother on a disability pension. He has often cited his upbringing as the foundation for his progressive beliefs.Mr Albanese believed his father had died before he was born, but as a teenager he learned his mother had in fact fallen pregnant to a married man - who was very likely still alive - while travelling in Europe.Mr Albanese credits his humble beginnings as the foundation for his progressive beliefs.Three decades later he tracked down Carlo Albanese, flying to Italy to meet his father for the first time, and his half-siblings.Mr Albanese said his mother, Maryanne Ellery, was determined to ensure he had opportunities that she never did. With her support, he became the first in his family to finish school and go to university.He has said creating a better world for his own son, Nathan, is the inspiration behind his public life. Mr Albanese separated from his wife of 19 years in 2019 but partner Jodie Haydon joined him on the campaign trail.
Mr Albanese has been the Labor Party leader for three years, taking over after the shock election defeat of predecessor Bill Shorten in 2019.But he has been a Labor Party stalwart since his 20s.He worked in both federal and state politics before he was elected on his 33rd birthday to an inner-city Sydney seat in 1996.In 2007, when Labor swept to power under Kevin Rudd, Mr Albanese became the minister for infrastructure and transport.He remained an influential figure as the party entered a tumultuous period after it replaced Mr Rudd with Julia Gillard in 2010.Anthony Albanese was instrumental in Kevin Rudd returning as prime minister in 2013When Mr Rudd reclaimed the prime ministership in 2013, Mr Albanese's backing saw him elevated to deputy prime minister. However he only held the position for 10 weeks because Labor lost the election.Mr Albanese then put himself forward to be head of the party. Despite being popular among rank-and-file party members, rival Bill Shorten had more support among members of parliament and got the job, becoming Australia's opposition leader.
But Mr Albanese's time finally came in 2019, after Mr Shorten lost two elections and was unseated as the Labor leader.
Australia's next PM
Mr Albanese has been a leading voice of the Labor Party's left faction, but since becoming leader he has positioned himself more towards the centre.In the lead up to the election he was at pains to prove he is "not woke" - an appeal to more conservative voters who abandoned the party at the 2019 poll.This has included walking back his support of more aggressive climate action policies while ramping up tougher rhetoric on China and national security.As leader, Mr Albanese has tried to appeal to more voters in the centre.He has also backed Australia's controversial policy to turn back any asylum seekers arriving by boat - something he once publicly opposed.Mr Albanese has undergone a physical transformation in recent years, losing weight and debuting a new wardrobe, crediting a car crash in 2021 as a wakeup call.
He stayed true to his Labor roots, however, in promising big spending on the country's troubled aged care sector, cheaper childcare, and reinvigorating the manufacturing industry.Labor has pledged to hold a referendum on enshrining in the constitution an Indigenous Voice to Parliament - an advisory body that would give Aboriginal and Torres Strait Islander people a role in shaping policies that affect them.Mr Albanese opened his victory speech by reiterating this promise.
"""

In [17]:
import shutil, os
from keybert import KeyBERT
from google_images_download import google_images_download
import numpy as np
import skvideo.io
import cv2
from transformers import pipeline

class generateVideo:
    def __init__(self,article):
        self.article = article

    def summarize(self):
        '''Summarizes the article'''
        summarizer = pipeline("summarization")
        text = summarizer(self.article, max_length=130, min_length=30, do_sample=False)[0]['summary_text']
        return text

    def dowloadImages(self,doc):
        '''Downloads images based on keywords and stores them in downloads folder and returns a list of key words and sentences'''
        kw_model = KeyBERT()
        response = google_images_download.googleimagesdownload()
        sentences = doc.split('.')
        key_word_list = []
        for sentence in sentences:
            keyword = kw_model.extract_keywords(sentence,keyphrase_ngram_range=(2, 2),top_n=1)
            if len(keyword)>0:
                arguments = {"keywords":keyword[0][0],"limit":1}
                paths = response.download(arguments)
                key_word_list.append(keyword[0][0])
        return key_word_list,sentences


    def overlayText(self,img_name,doc):
        '''Overlays text on an image'''
        img = cv2.imread(img_name)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB )
        try:
            img = cv2.resize(img, (800, 800))
        except:
            img = img[:800,:800,:]
        # Defining the text parameters
        font                   = cv2.FONT_HERSHEY_SIMPLEX
        fontScale              = 1
        fontColor              = (255,255,255)
        thickness              = 2
        lineType               = 2
        sentences = []
        # Splitting each sentence to multiple lines
        for cnt,letter in enumerate(doc):
            sentence = doc[:45]
            sentences.append(sentence)
            doc = doc[45:]
            if len(doc) < 45:
                sentences.append(doc)
                break
        # Creating a bottom border on the image for clear visibilty of overlayed text depending on the length of text
        border = 800 - 50*len(sentences)
        img[border:,:,:] = [125,125,125]
        # Overlaying text line by line in  an image
        for line_no in range(len(sentences)):
            start_x = 10 + 3*line_no
            start_y = 800 - 28*(len(sentences) - line_no)
            location = (start_x,start_y)
            img = cv2.putText(img,sentences[line_no],location,font, fontScale,fontColor,thickness,lineType)   
        return img

    def createVideo(self,doc):
        '''Stiches the images in the images folder and makes a video by overlaying appropriate text on top of them'''
        # Defining height and width of the video
        height = 800
        width = 800
        start = 0
        end = 125
        
        key_word_list,sentences = self.dowloadImages(doc)
        out_video =  np.empty([125*len(key_word_list), height, width, 3], dtype = np.uint8)
        out_video =  out_video.astype(np.uint8)

        for cnt,key_word in enumerate(key_word_list):
            os.chdir('/content/downloads'+'/'+key_word)
            for img_name in os.listdir():
                img = self.overlayText(img_name,sentences[cnt])
                out_video[start:end] = img
                start += 125
                end += 125
                
        # Writes the the output image sequences in a video file
        os.chdir('/content')
        skvideo.io.vwrite("video.mp4", out_video)
        shutil.rmtree('/content/downloads')
        print('Video created!!')
        


In [18]:
text =  generateVideo(article)
summary = text.summarize()
text.createVideo(summary)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)



Item no.: 1 --> Item name = anthony albanese
Evaluating...
Starting Download...
Completed Image ====> 1.1267.jpg

Errors: 0


Item no.: 1 --> Item name = sydney seat
Evaluating...
Starting Download...
Completed Image ====> 1.maxresdefault.jpg

Errors: 0


Item no.: 1 --> Item name = shorten 2019
Evaluating...
Starting Download...
Completed Image ====> 1.5568.jpg

Errors: 0


Item no.: 1 --> Item name = mr albanese
Evaluating...
Starting Download...
Completed Image ====> 1.australian_labor_party_leader_anthony_albanese_mp_%28cropped_-_tight%29.jpg

Errors: 0

Video created!!
